# 🤖 自動研究実行システム - Colab版

**ワンクリックで研究を自動実行**するColabノートブックです。

In [ ]:
# 🚀 自動実行トリガー - このセルを実行するだけ！

AUTO_RUN = True  # これをTrueにするだけで全自動実行
UPLOAD_IMAGES = True  # 画像アップロードも自動化
SAVE_TO_DRIVE = True  # Google Driveに自動保存

print("🎯 自動研究システムを開始します...")
print(f"📊 自動実行: {'有効' if AUTO_RUN else '無効'}")
print(f"📸 画像アップロード: {'自動' if UPLOAD_IMAGES else '手動'}")
print(f"💾 Drive保存: {'有効' if SAVE_TO_DRIVE else '無効'}")

In [ ]:
# 環境自動セットアップ
if AUTO_RUN:
    print("🔧 環境を自動セットアップ中...")
    
    # Colab環境確認
    try:
        import google.colab
        IN_COLAB = True
        print("✅ Google Colab環境確認")
    except:
        IN_COLAB = False
        print("❌ ローカル環境で実行")
    
    # 必要なライブラリを一括インストール
    if IN_COLAB:
        import subprocess
        import sys
        
        packages = [
            'torch', 'torchvision', 'torchaudio',
            'transformers', 'ultralytics', 'opencv-python',
            'pillow', 'matplotlib', 'seaborn', 'pandas',
            'numpy', 'nltk', 'scipy', 'scikit-learn'
        ]
        
        for package in packages:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
        
        print("✅ ライブラリインストール完了")
    
    # Google Drive自動マウント
    if SAVE_TO_DRIVE and IN_COLAB:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        print("✅ Google Drive自動接続完了")
    
    print("🎉 自動セットアップ完了！")
else:
    print("⏸️ 手動モード - 必要に応じてセットアップしてください")

In [ ]:
# 研究システム自動初期化
if AUTO_RUN:
    print("🤖 AI研究システムを自動初期化中...")
    
    # 必要なライブラリインポート
    import torch
    import torch.nn as nn
    from transformers import BlipProcessor, BlipForConditionalGeneration
    from transformers import CLIPProcessor, CLIPModel
    from ultralytics import YOLO
    import cv2
    from PIL import Image
    import matplotlib.pyplot as plt
    import pandas as pd
    import numpy as np
    import json
    import nltk
    import seaborn as sns
    from scipy import stats
    import os
    import warnings
    warnings.filterwarnings('ignore')
    
    # デバイス自動設定
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"🔥 使用デバイス: {device}")
    
    # NLTK自動ダウンロード
    nltk.download('wordnet', quiet=True)
    nltk.download('omw-1.4', quiet=True)
    from nltk.corpus import wordnet as wn
    
    # AIモデル自動初期化
    print("🧠 AIモデルを読み込み中...")
    blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
    
    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
    
    yolo_model = YOLO('yolov8n.pt')
    
    print("✅ AI研究システム初期化完了！")
else:
    print("⏸️ 手動モード - AIモデルを手動で初期化してください")

In [ ]:
# 自動研究システムクラス
if AUTO_RUN:
    class AutoResearchSystem:
        def __init__(self):
            self.results = []
            self.categories = {
                'person': ['person', 'man', 'woman', 'child', 'people', 'human', 'face'],
                'animal': ['dog', 'cat', 'bird', 'horse', 'cow', 'elephant', 'bear'],
                'food': ['pizza', 'hamburger', 'cake', 'apple', 'banana', 'sandwich'],
                'vehicle': ['car', 'bus', 'truck', 'motorcycle', 'bicycle', 'airplane'],
                'building': ['house', 'building', 'bridge', 'castle', 'church'],
                'furniture': ['chair', 'table', 'sofa', 'bed', 'desk'],
                'plant': ['tree', 'flower', 'grass', 'plant', 'garden'],
                'landscape': ['mountain', 'beach', 'lake', 'river', 'sky']
            }
            print("🔬 自動研究システム準備完了")
        
        def auto_analyze_image(self, image_path):
            """画像を自動分析"""
            try:
                # 画像読み込み
                image = Image.open(image_path).convert('RGB')
                
                # 1. キャプション自動生成
                inputs = blip_processor(image, return_tensors="pt").to(device)
                out = blip_model.generate(**inputs, max_length=50)
                caption = blip_processor.decode(out[0], skip_special_tokens=True)
                
                # 2. 意味カテゴリ自動検出
                caption_lower = caption.lower()
                detected_category = 'general'
                max_score = 0
                
                for category, keywords in self.categories.items():
                    score = sum(1 for keyword in keywords if keyword in caption_lower)
                    if score > max_score:
                        max_score = score
                        detected_category = category
                
                # 3. 分類実験自動実行
                if detected_category in self.categories:
                    labels = self.categories[detected_category]
                    
                    # 汎用アプローチ
                    general_inputs = clip_processor(text=labels, images=image, return_tensors="pt", padding=True).to(device)
                    with torch.no_grad():
                        general_outputs = clip_model(**general_inputs)
                        general_probs = general_outputs.logits_per_image.softmax(dim=1)
                        general_confidence = general_probs.max().item()
                        general_label = labels[general_probs.argmax().item()]
                    
                    # 特化アプローチ
                    specialized_labels = [f"a photo of {detected_category}: {label}" for label in labels]
                    spec_inputs = clip_processor(text=specialized_labels, images=image, return_tensors="pt", padding=True).to(device)
                    with torch.no_grad():
                        spec_outputs = clip_model(**spec_inputs)
                        spec_probs = spec_outputs.logits_per_image.softmax(dim=1)
                        specialized_confidence = spec_probs.max().item()
                        specialized_label = labels[spec_probs.argmax().item()]
                    
                    # 改善率計算
                    improvement = (specialized_confidence - general_confidence) / general_confidence * 100
                else:
                    general_label = general_confidence = specialized_label = specialized_confidence = None
                    improvement = 0
                
                # 結果記録
                result = {
                    'image': image_path,
                    'caption': caption,
                    'category': detected_category,
                    'general_label': general_label,
                    'general_confidence': general_confidence,
                    'specialized_label': specialized_label,
                    'specialized_confidence': specialized_confidence,
                    'improvement_rate': improvement
                }
                
                self.results.append(result)
                return result
                
            except Exception as e:
                print(f"❌ エラー: {e}")
                return None
        
        def auto_visualize_results(self):
            """結果を自動可視化"""
            if not self.results:
                print("❌ 分析結果がありません")
                return
            
            df = pd.DataFrame(self.results)
            
            # 統計表示
            valid_improvements = df[df['improvement_rate'] != 0]['improvement_rate']
            print(f"\n📊 自動分析結果:")
            print(f"  処理画像数: {len(df)}")
            print(f"  平均改善率: {valid_improvements.mean():.2f}%")
            print(f"  最大改善率: {valid_improvements.max():.2f}%")
            
            # 自動グラフ生成
            plt.figure(figsize=(12, 8))
            
            # 改善率分布
            plt.subplot(2, 2, 1)
            plt.hist(valid_improvements, bins=10, alpha=0.7, color='skyblue')
            plt.title('改善率分布')
            plt.xlabel('改善率 (%)')
            
            # カテゴリ別改善率
            plt.subplot(2, 2, 2)
            category_improvement = df[df['improvement_rate'] != 0].groupby('category')['improvement_rate'].mean()
            category_improvement.plot(kind='bar', color='lightgreen')
            plt.title('カテゴリ別改善率')
            plt.xticks(rotation=45)
            
            # カテゴリ分布
            plt.subplot(2, 2, 3)
            df['category'].value_counts().plot(kind='pie', autopct='%1.1f%%')
            plt.title('カテゴリ分布')
            
            # 確信度比較
            plt.subplot(2, 2, 4)
            valid_data = df[(df['general_confidence'].notna()) & (df['specialized_confidence'].notna())]
            plt.scatter(valid_data['general_confidence'], valid_data['specialized_confidence'])
            plt.plot([0, 1], [0, 1], 'r--')
            plt.title('確信度比較')
            plt.xlabel('汎用アプローチ')
            plt.ylabel('特化アプローチ')
            
            plt.tight_layout()
            plt.show()
            
            return df
        
        def auto_save_results(self):
            """結果を自動保存"""
            if not self.results:
                return
            
            # CSV保存
            df = pd.DataFrame(self.results)
            df.to_csv('auto_research_results.csv', index=False)
            
            # JSON保存
            with open('auto_research_results.json', 'w', encoding='utf-8') as f:
                json.dump(self.results, f, ensure_ascii=False, indent=2)
            
            print("✅ 結果を自動保存しました")
            
            # Colab自動ダウンロード
            if IN_COLAB:
                from google.colab import files
                files.download('auto_research_results.csv')
                files.download('auto_research_results.json')
                print("📥 ファイルを自動ダウンロードしました")
            
            # Google Drive自動バックアップ
            if SAVE_TO_DRIVE and IN_COLAB and os.path.exists('/content/drive/MyDrive'):
                import shutil
                backup_dir = '/content/drive/MyDrive/auto_research_backup'
                os.makedirs(backup_dir, exist_ok=True)
                shutil.copy('auto_research_results.csv', backup_dir)
                shutil.copy('auto_research_results.json', backup_dir)
                print("☁️ Google Driveに自動バックアップしました")
    
    # 研究システム初期化
    research_system = AutoResearchSystem()
    print("🎯 自動研究システム準備完了！")
else:
    print("⏸️ 手動モード")

In [ ]:
# 🚀 メイン自動実行セル - ここだけ実行すれば全て完了！

if AUTO_RUN:
    print("🎯 自動研究を開始します！")
    print("="*50)
    
    # 1. 画像自動アップロード
    if UPLOAD_IMAGES and IN_COLAB:
        print("📸 研究用画像をアップロードしてください:")
        from google.colab import files
        uploaded = files.upload()
        image_files = list(uploaded.keys())
        print(f"✅ {len(image_files)}個の画像をアップロードしました")
    else:
        # サンプル画像（実際の使用時は削除）
        image_files = []  # 空の場合はサンプル使用
        print("⚠️ 画像がアップロードされていません")
    
    # 2. 自動画像分析
    if image_files:
        print(f"\n🔬 {len(image_files)}個の画像を自動分析中...")
        for i, image_file in enumerate(image_files):
            print(f"📊 分析中 ({i+1}/{len(image_files)}): {image_file}")
            result = research_system.auto_analyze_image(image_file)
            if result:
                print(f"  ✅ カテゴリ: {result['category']}")
                print(f"  📈 改善率: {result['improvement_rate']:.2f}%")
        
        # 3. 結果自動可視化
        print("\n📊 結果を自動可視化中...")
        df = research_system.auto_visualize_results()
        
        # 4. 結果自動保存
        print("\n💾 結果を自動保存中...")
        research_system.auto_save_results()
        
        # 5. 最終サマリー
        print("\n" + "="*50)
        print("🎉 自動研究完了！")
        print("="*50)
        
        valid_improvements = df[df['improvement_rate'] != 0]['improvement_rate']
        print(f"📊 最終結果:")
        print(f"  - 処理画像数: {len(df)}")
        print(f"  - 検出カテゴリ数: {df['category'].nunique()}")
        if len(valid_improvements) > 0:
            print(f"  - 平均改善率: {valid_improvements.mean():.2f}%")
            print(f"  - 最大改善率: {valid_improvements.max():.2f}%")
            if valid_improvements.mean() > 5:
                print(f"  🚀 特化アプローチが有効です！")
            else:
                print(f"  📝 改善の余地があります")
        
        print(f"\n📥 結果ファイル:")
        print(f"  - auto_research_results.csv (自動ダウンロード済み)")
        print(f"  - auto_research_results.json (自動ダウンロード済み)")
        if SAVE_TO_DRIVE:
            print(f"  - Google Drive バックアップ完了")
    
    else:
        print("❌ 分析する画像がありません")
        print("📸 上のセルで画像をアップロードしてください")

else:
    print("⏸️ 自動実行が無効です")
    print("🔧 AUTO_RUN = True に設定して実行してください")

## 🎯 使用方法

### 簡単3ステップ！

1. **最初のセル実行**: `AUTO_RUN = True` を確認
2. **画像アップロード**: 研究用画像を選択
3. **自動実行完了**: 結果が自動でダウンロード

### 設定オプション
```python
AUTO_RUN = True      # 全自動実行
UPLOAD_IMAGES = True # 画像アップロード
SAVE_TO_DRIVE = True # Google Drive保存
```

### 出力ファイル
- `auto_research_results.csv` - 詳細データ
- `auto_research_results.json` - JSON形式
- Google Drive自動バックアップ

---
**🚀 ワンクリックで研究完了！**